In [42]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import random

## Evidence Appx Equations

$$ \alpha = \frac{\gamma}{\bold{m}_N^T\bold{m}_N} \tag{3.92}$$

$$ \frac{1}{\beta} = \frac{1}{N-\gamma}\sum_{n=1}^N\{ t_n - \bold{m}_N^T \phi(\bold{x}_n)\}^2 \tag{3.95} $$


where $\Phi$ is the design matrix, $\bold{A} = \alpha I + \beta \Phi^T \Phi$, and
$$ \bold{m}_N = \beta \bold{A}^{-1} \bold{\Phi}^T \bold{\text{t}}. \tag{3.84}$$

We will use the eigenvector equation

$$ (\beta \Phi^T \Phi) \bold{u}_i = \lambda_i \bold{u}_i \tag{3.87}$$


 to find a new $\gamma$ through this equation

$$\gamma = \sum_i \frac{\lambda_i}{\alpha + \lambda_i} \tag{3.91}$$

**Procedure:**
1. Choose inital $\alpha$ and $\beta$.
2. Calculate $\bold{m}_N$ (3.84) and $\gamma$ (3.91).
3. Re-estimate $\alpha$ and $\beta$.
4. Repeat steps 2 and 3 until convergence.

In [69]:
def evidence_approx(X, y, alpha, beta, M):
    N = len(X)
    phi = np.ones((N, M))
    phi[:, 1] = X
    
    # Calculate mN 
    A = alpha * np.eye(M) + beta * np.dot(phi.T, phi) 
    mN = beta * np.dot(np.linalg.inv(A), np.dot(phi.T, y)) # Eq. 3,84

    # Calculate gamma 
    eig = beta * np.dot(phi.T, phi) # Eq. 3.87
    eig_val, eig_vec = np.linalg.eig(eig)

    gamma =  0 # Eq. 3.91
    for i in range(eig_val.shape[0]):
        gamma += eig_val[i] / (alpha + eig_val[i])
    
    pred_alpha = gamma / np.dot(mN.T, mN) # Eq. 3.92

    sum = 0
    for n in range(N):   # sum of 3.95 to find convergence
        sum += (y[n] - np.dot(mN.T, phi[n, :])) ** 2
    pred_beta = 1 / (N - gamma) * sum   # Eq. 3.95
    pred_beta = 1 / pred_beta

    return pred_alpha, pred_beta

In [73]:
np.random.seed(42)
def generate_data(beta, X):
    N = len(X)
    noise = np.random.normal(0, np.sqrt(1/beta), N)
    y = np.sin(2*np.pi*X) + noise
    return y

In [86]:
size = 100
alpha = 2
beta = 2

X = np.random.uniform(-1, 1, size)
y = generate_data(beta, X)

In [94]:
# initialize alphas and betas
pred_alpha = 0.5
pred_beta = 1
iters = 0

while True:
    new_alpha, new_beta = evidence_approx(X, y, pred_alpha, pred_beta, 2)
    if abs(pred_alpha - new_alpha) < 1e-3 and abs(pred_beta - new_beta) < 1e-3:
        break
    pred_alpha = new_alpha
    pred_beta = new_beta
    iters+=1
print(f'Iters: {iters}, Predicted alpha & beta: {np.round(new_alpha,2), np.round(new_beta,2)}')

Iters: 5, Predicted alpha & beta: (7.77, 2.3)
